## Разбиение текста на предложения (Определение границ предложений)


Сегментация предложений - базовая задача в NLP. Даже самые сложные модели пока плохо работают с длинными последовательностями, поэтому если нужно решать что-то сложнее сентимент анализа (например, выделять сущности), то первым делом текст нужно поделить на предложения.

От того, в какой форме у нас текст, зависит сложность задачи. Она может быть либо почти тривиальной, либо крайне сложной.

###  1. Текст в стандартной форме

Если в тексте есть знаки препинания, соблюдается регистр и мало ошибок -  найти границы предложений для большинства случаев будет достаточно просто. Большинство предложений заканчиваются на точку, вопросительный или восклицательные знаки, после которых идет какой-то отступ и другое слово с заглавной буквы. 

Да иногда будут попадаться аббревиатуры и сокращения или случаи, когда пробела после точки не стоит, но для задач, где не нужна точность - это не будет сильной проблемой.

Посмотрим, как оно работает на небольшом корпусе новостных текстов. Скачайте его отсюда - https://github.com/mannefedov/ru_kw_eval_datasets/blob/master/data/ng_0.jsonlines.zip

In [34]:
import os
from nltk import sent_tokenize
import re
import pandas as pd

In [35]:
# у меня данные лежат на уровень выше в папке data
files= ['../data/'+file for file in os.listdir('../data')]
data = pd.concat([pd.read_json(file, lines=True) for file in files], axis=0, ignore_index=True)['content'].tolist()

Для примера возьмем совсем немного текстов.

In [36]:
data = data[:30]

Каждая статья - просто какой-то текст.

In [37]:
data[0][:1000]

'Многие интересуются, зачем нужна «Яблоку» молодежная фракция? Основной задачей «Молодежного «Яблока» является привлечение молодых людей к участию в выборах и деятельности партии. «Молодежное «Яблоко» работает более чем в 10 регионах. Единого руководства у нас нет, но мы стараемся координировать свою деятельность и периодически проводим акции на федеральном уровне.\nМы ведем борьбу с обязательным воинским призывом. Военный – это профессия, а не обязанность. Молодые люди вправе сами распоряжаться своей жизнью и не терять целый год, отдавая государству «долг», который они у него не занимали. По мнению одного из ведущих специалистов в области оборонной политики Алексея Арбатова, переход на контрактную армию будет стоить лишь 2% военного бюджета.\nТакже на федеральном уровне «Молодежное «Яблоко» проводило акции за освобождение политзаключенных и против вмешательства России во внутреннюю политику Украины.\nРасскажу о московских активистах. Виктору Петрунину – 19 лет, он пришел к нам больше 

Для начала давайте просто попробуем разбить предложения простой регуляркой, которая будет делить предложения по **.?!**, последующему пробелу и заглавной букве.

In [40]:
regex = '' # напишите регулярку тот

In [41]:
re.split(regex, data[0])[:5] # вот так должно работать

['Многие интересуются, зачем нужна «Яблоку» молодежная фракция?',
 'Основной задачей «Молодежного «Яблока» является привлечение молодых людей к участию в выборах и деятельности партии. «Молодежное «Яблоко» работает более чем в 10 регионах.',
 'Единого руководства у нас нет, но мы стараемся координировать свою деятельность и периодически проводим акции на федеральном уровне.',
 'Мы ведем борьбу с обязательным воинским призывом.',
 'Военный – это профессия, а не обязанность.']

Видно, что даже таким простым способом правильно разбивается бОльшая часть текста. Ошибку появляются, когда текст начинается с числа или слова в кавычках. Их, конечно, можно добавить в регулярку, но, скорее всего, это приведет к появлению других ошибок (`руб. 2`). Да и для того, что мы написали несложно придумать контрпример. Поэтому если уж решили усложнять, то нужно подойди к этому более подробно и разобраться с сокращениями.

Ну или не разбираться, а взять уже готовое!

В nltk как раз есть такая штука. Так как в русском и английском знаки препинания по большей части одинаковые - можно смело ей пользоваться прямо из коробки.

In [42]:
[re.sub('[\n\t]', ' ', x) for x in sent_tokenize(data[0])[:30]]

['Многие интересуются, зачем нужна «Яблоку» молодежная фракция?',
 'Основной задачей «Молодежного «Яблока» является привлечение молодых людей к участию в выборах и деятельности партии.',
 '«Молодежное «Яблоко» работает более чем в 10 регионах.',
 'Единого руководства у нас нет, но мы стараемся координировать свою деятельность и периодически проводим акции на федеральном уровне.',
 'Мы ведем борьбу с обязательным воинским призывом.',
 'Военный – это профессия, а не обязанность.',
 'Молодые люди вправе сами распоряжаться своей жизнью и не терять целый год, отдавая государству «долг», который они у него не занимали.',
 'По мнению одного из ведущих специалистов в области оборонной политики Алексея Арбатова, переход на контрактную армию будет стоить лишь 2% военного бюджета.',
 'Также на федеральном уровне «Молодежное «Яблоко» проводило акции за освобождение политзаключенных и против вмешательства России во внутреннюю политику Украины.',
 'Расскажу о московских активистах.',
 'Виктору Петру

Ни одной ошибки!

Если уточнить разбиения все-таки нужно, есть способ обучить токенизатор на имеющихся текстах или даже добавить туда спорные случаи вручную.

Про то как алгоритм обучается можно почитать тут  - http://www.aclweb.org/anthology/J06-4003  
В двух словах: считаются частотные аббревиатуры, которые потом не используются как разделители.


In [43]:
from pprint import pprint
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktTrainer

In [44]:
trainer = PunktTrainer()
trainer.INCLUDE_ALL_COLLOCS = True
trainer.train('\n'.join(data))
 
tokenizer = PunktSentenceTokenizer(trainer.get_params())

Список выученных сокращений можно достать вот так:

In [45]:
print(tokenizer._params.abbrev_types)

{'долл', 'т.д', 'ю.н', 'см', 's', 'с.г', 'аэс', 'д', 'г', 'б', 'кв', 'пер', 'н.а', 'пл', 'др', 'куб', 'тыс', 'руб', 'стр', 'я»', 'м»', 'н.к'}


Чтобы удобнее просматривать окончания можно отрезать все до последних 10 символов:

In [46]:
[re.sub('[\n\t]', ' ', x)[-10:] for x in tokenizer.tokenize(data[5])[:15]]

['е (Техас).',
 'оехали 15.',
 'ить заезд.',
 ' Росберга.',
 'Риккъярдо.',
 'е удалась.',
 'це заезда.',
 ' досрочно.',
 ' Marussia.',
 ' Абу-Даби.',
 'авершению.',
 'ио Переса.',
 'айкконена.',
 'до боксов.',
 'щей гонки.']

Ну и в него же напрямую можно что-то добавить (без учета регистра и без точки на конце)

In [47]:
# можно сразу добавить все числа и сокращения имен и отчеств
tokenizer._params.abbrev_types.add('15')

In [48]:
[re.sub('[\n\t]', ' ', x)[-10:] for x in tokenizer.tokenize(data[5])[:15]]

['е (Техас).',
 'ить заезд.',
 ' Росберга.',
 'Риккъярдо.',
 'е удалась.',
 'це заезда.',
 ' досрочно.',
 ' Marussia.',
 ' Абу-Даби.',
 'авершению.',
 'ио Переса.',
 'айкконена.',
 'до боксов.',
 'щей гонки.',
 'кий сход).']

После нескольких итераций ручной подкрутки может возникнуть желание более точно измерить качество разбиений.

Как это можно измерить?

Можно взять какой-то текст и разбить его руками. А потом разбить этот ж текст моделью и сравнить длины полученных списков.

In [49]:
gold = tokenizer.tokenize(data[3]) # руками мы разбивать конечно не будем, а просто проверим, что все правильно

In [50]:
gold

['В Ленинском районном суде продолжаются слушания по делу экс-депутата Думы Владивостока Зинаиды Ким и бывшего председателя избирательного участка № 522 Елены Футиной, которых обвиняют в сговоре и фальсификациях результатов на выборах на сентябрьских выборах 2016 года.',
 'Напомним, 18 сентября 2017 года местные журналисты сняли на видео, как Ким, будучи кандидатом по спискам в Законодательное Собрание Приморского края, выдавала молодым людям открепительные, возила их голосовать на участок, где уже знали о предстоящем визите.',
 'В качестве вознаграждения избирателям предлагалось по 500 рублей.',
 'Перед началом судебного процесса Зинаида Ким разговаривала с журналистами на повышенных тонах и обзывая, доказывала, что видео – монтаж.',
 'Адвокаты представили вниманию участников процесса характеристику подсудимой, составленную руководителями Всероссийской общественной организации «Боевое братство (Приморье)», членом которого является подсудимая.',
 'Выяснилось, что у Зинаиды Ким – богаты

In [51]:
my_split = sent_tokenize(data[3])

In [52]:
print('Gold length - ', len(gold))
print('My split - ', len(my_split))

Gold length -  19
My split -  21


Понятно, что это не очень хороший способ (его можно обмануть), но что-то он покажет.

Также можно привести списки к множествам и посчитать пересечения - одинаковые предложения будут считать совпадающими элементами.

Тут нужно быть аккуратным, так как модель может отрезать один лишний символ и это уже будет считаться промахом.

In [53]:
# добавим нормировку на длину объединения
len(set(gold) & set(my_split)) / len(set(gold) | set(my_split))

0.8181818181818182

Есть ещё такая метрика - берем идеальное предложение и пробуем разбить его нашей моделью - если  разбивается, считает этой ошибкой, если нет - правильным ответом. 
Потом берем два идеальных предложения - склеиваем их пробельным символом и пробуем разбить нашей моделью. Если разбивает на два нужных предложения - все правильно.

Таким образом мы вычисляем tp fp fn для расчета точности, полноты и f1 меры.

In [54]:
tp = 0
fp = 0
fn = 0

for sent in gold:
    if len(sent_tokenize(sent)) == 1:
        tp += 1
    else:
        fp += 1

for i in range(len(gold)-1):
    sent1, sent2 = gold[i], gold[i+1]
    sent = ' '.join([sent1, sent2])
    if len(sent_tokenize(sent)) == 2:
        tp += 1
    else:
        fn += 1

precision = (tp/(tp+fp))
recall = (tp/(tp+fn))
f1 = 2*(precision*recall)/(precision+recall)
print('Precision - ', precision)
print('Recall - ', recall)
print('F1 - ', f1)

Precision -  0.9714285714285714
Recall -  0.9444444444444444
F1 -  0.9577464788732395


Такую метрики использовали авторы вот этого токенизатора для русского - https://github.com/deepmipt/ru_sentence_tokenizer

Можно придумать и что-то посложнее. 

Перейдем к индексам символов в тексте. Теперь нам нужно для каждого символа предсказать является ли он разбивающим или нет. Потом можно применять стандартные метрики качества классификации.

In [55]:
tokenizer.span_tokenize(data[3])

[(0, 267),
 (268, 525),
 (526, 591),
 (592, 728),
 (729, 939),
 (940, 1164),
 (1166, 1445),
 (1446, 1566),
 (1568, 1757),
 (1758, 1897),
 (1898, 1981),
 (1982, 2085),
 (2086, 2233),
 (2234, 2318),
 (2319, 2463),
 (2464, 2578),
 (2579, 2726),
 (2727, 3013),
 (3015, 3026)]

In [56]:
gold = [0 for i in range(len(data[3]))]
for span in tokenizer.span_tokenize(data[3]):
    gold[span[1]-1] = 1


In [57]:
len(sent_tokenize(data[3])[0])

267

In [58]:
my_split = [0 for i in range(len(data[3]))]
index = 0
for sent in sent_tokenize(data[3]):
    index += len(sent)
    my_split[index] = 1

In [59]:
from sklearn.metrics import classification_report

In [60]:
print(classification_report(gold, my_split))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      3007
           1       0.05      0.05      0.05        19

   micro avg       0.99      0.99      0.99      3026
   macro avg       0.52      0.52      0.52      3026
weighted avg       0.99      0.99      0.99      3026



Ещё можно выписать индексы всех разбивающих символов и рассматривать это как строку. Между двумя строками (идеальной и той, что выдала модель) можно посчитать edit distance.

In [61]:
import editdistance

In [62]:
editdistance.eval([1,2,3], [1,2,3,4])

1

In [63]:
gold = []
my_split = []
for span in tokenizer.span_tokenize(data[3]):
    gold.append(span[1])

index = 0
for sent in sent_tokenize(data[3]):
    index += len(sent)
    my_split.append(index)

In [64]:
editdistance.eval(gold, my_split)

20

Вот тут можно почитать про другие метрики (и их сравнение) - http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.417.8097&rep=rep1&type=pdf

### 2. Нестандартный текст

Бывает, что текст приходит откуда-то ещё (например, с speech2text модуля) сплошняком без каких-либо знаков препинания и регистров. Либо в нём столько неточностей, что стандартные методы ошибаются на каждом втором слове (например, при переводе пдфа в текст).

В этом случае нужно как-то учитывать смысл написанного. 

Для этого хорошо подойдут рекурентные нейронные сети. 

Обучающую выборку (если её нет) можно попробовать собрать из обычных текстов, разделенных вручную или автоматически, убрав всю пунктуацию и приведя их к нижнему регистру. Ну или сделать то же самое с предложениями из какого-нибудь синтагруса.

Саму задачу можно рассматривать как задачу классификации. К целым предложениям можно приписать положительный класс (т.е. после последнего слова нужно поставить разделитель), а для подбора отрицательного класса, можно нарезать предложения на части и каждой из них приписать отрицательный класс (т.е. разделителя не нужно). 

Например, предложение "вчера кажется бы снег" преборазуется в:

"вчера кажется был снег" - 1
"вчера кажется был" - 0
"кажется был" - 0
"вчера кажется" - 0

Вообще можно и положительный класс пополнить таким методом (выбрость первое слово, например), но лучше этого не делать. Потому что предложения бывают разные и не всегда можно что-то обросить без потери смысла. В нулевой класс тоже по идее могут попасть нормальные предложения (был снег, например). Но можно предположить, что такое будет случаться не часто.

Давайте попробуем на небольшом количестве предложений.

**Если вы ещё ничего не знаете про нейронные сети или знаете очень мало, не пугайтесь.** Попробуйте запустить мой код и посмотреть, что он делает. Попробуйте что-то поменять и посмотреть что изменится. Ну и потом уже можете пойти почитать про то, как оно устроено. Такой top-down (от практики к теории) используют в fast.ai для обучения нейронным сетям и это работает!  
  https://www.fast.ai/2016/10/08/teaching-philosophy/

Главное, что нужно знать про рекуррентные нейронные сети (LSTM в нашем случае) - то, что они работают с последовательностью. На каждом шаге учитывается накопленная информация с предыдущих шагов и текущий шаг. На последнем шаге получается сжатое представление всей последовательности, по которому уже и делается предсказание. 

Если у вас нет нужных библиотек (keras) - установите их через pip. Если они не устанавливаются - попробуйте открыть тетрадку в Colab (там все установлено сразу).

In [81]:
from string import punctuation
import numpy as np
from collections import Counter
from keras.layers import LSTM, Embedding, Dense, Dropout
from keras import Sequential
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences

punct=punctuation+'«…»'
def normalize(text):
    tokens = [word.strip(punct) for word in text.lower().split()]
    tokens = [word for word in tokens if word]
    return tokens

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [66]:
normalize(tokenizer.tokenize(data[3])[0])[:10]

['в',
 'ленинском',
 'районном',
 'суде',
 'продолжаются',
 'слушания',
 'по',
 'делу',
 'экс-депутата',
 'думы']

Теперь возьмем все данные.

In [67]:
files= ['../data/'+file for file in os.listdir('../data')]
data = pd.concat([pd.read_json(file, lines=True) for file in files], axis=0, ignore_index=True)['content'].tolist()

In [68]:
good_sents = []
for text in data:
    sents = [normalize(sent) for sent in tokenizer.tokenize(text)]
    for sent in sents:
        if sent:
            good_sents.append(sent)

In [69]:
good_sents[:3]

[['многие',
  'интересуются',
  'зачем',
  'нужна',
  'яблоку',
  'молодежная',
  'фракция'],
 ['основной',
  'задачей',
  'молодежного',
  'яблока',
  'является',
  'привлечение',
  'молодых',
  'людей',
  'к',
  'участию',
  'в',
  'выборах',
  'и',
  'деятельности',
  'партии'],
 ['молодежное', 'яблоко', 'работает', 'более', 'чем', 'в', '10', 'регионах']]

In [70]:
bad_sents = []
for sent in good_sents:
    not_lust = sent[:-1]
    while len(not_lust) >= 1:
        bad_sents.append(not_lust)
        not_lust = not_lust[:-1]
    

Посмотрим, что нарезалось.

In [71]:
good_sents[0][:-1]

['многие', 'интересуются', 'зачем', 'нужна', 'яблоку', 'молодежная']

In [72]:
bad_sents[:4]

[['многие', 'интересуются', 'зачем', 'нужна', 'яблоку', 'молодежная'],
 ['многие', 'интересуются', 'зачем', 'нужна', 'яблоку'],
 ['многие', 'интересуются', 'зачем', 'нужна'],
 ['многие', 'интересуются', 'зачем']]

Теперь нам нужно сделать словарик и заменить слово на его индекс.

In [73]:
sents = good_sents + bad_sents
target = [1 for i in range(len(good_sents))] + [0 for i in range(len(bad_sents))]


In [74]:
vocab = Counter()
for sent in sents:
    vocab.update(sent)
    

In [75]:
len([x for x in vocab if vocab[x] > 5])

115476

In [76]:
id2word = {i+2:word for i, word in enumerate(vocab) if vocab[word] > 5}
word2id = {word:i for i, word in id2word.items()}

In [77]:
sents_ids = []

for sent in sents:
    sents_ids.append([word2id.get(word, 1) for word in sent])

In [78]:
sents_ids[:3]

[[92853, 11199, 48169, 136277, 95231, 76645, 125552],
 [49529,
  3675,
  123150,
  129445,
  64529,
  93068,
  99540,
  92004,
  104435,
  78303,
  66847,
  126384,
  47361,
  44636,
  135546],
 [6110, 136226, 116112, 17094, 79225, 66847, 100479, 125782]]

Последовательности должны быть одной длины. Стандартный подход - найти максимальную длину и дотянуть все другие предложения до этой длины, подставив, где нужно нули (это называется паддинг).

Но чтобы уменьшить модель можно взять среднюю длинну и обрезать все, что длиннее.

In [82]:
np.mean([len(s) for s in sents_ids])

12.098569722474977

In [83]:
X = pad_sequences(sents_ids, 20, truncating='pre') # возьмем 20, чтобы покрытыть побольше случаев

Теперь мы готовы обучаться!

In [84]:
maxlen = 20

In [85]:
model = Sequential()
model.add(Embedding(input_dim=len(vocab)+2, output_dim=32, input_length=maxlen))
model.add(Dropout(0.3))
model.add(LSTM(20))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [86]:
X_train, X_valid, y_train, y_valid = train_test_split(X, target, random_state=42)

In [87]:
model.fit(X_train, y_train, batch_size=128, epochs=10,
          validation_data=[X_valid, y_valid])

Train on 1266048 samples, validate on 422016 samples
Epoch 1/10
1266048/1266048 [==============================] - 282s 223us/step - loss: 0.1691 - acc: 0.9423 - val_loss: 0.1536 - val_acc: 0.9441
Epoch 2/10
1266048/1266048 [==============================] - 283s 223us/step - loss: 0.1326 - acc: 0.9491 - val_loss: 0.1486 - val_acc: 0.9448
Epoch 3/10
1266048/1266048 [==============================] - 302s 238us/step - loss: 0.1123 - acc: 0.9572 - val_loss: 0.1475 - val_acc: 0.9466
Epoch 4/10
1266048/1266048 [==============================] - 291s 230us/step - loss: 0.0978 - acc: 0.9632 - val_loss: 0.1501 - val_acc: 0.9470
Epoch 5/10
1266048/1266048 [==============================] - 289s 228us/step - loss: 0.0873 - acc: 0.9675 - val_loss: 0.1512 - val_acc: 0.9468
Epoch 6/10
1266048/1266048 [==============================] - 287s 227us/step - loss: 0.0793 - acc: 0.9706 - val_loss: 0.1589 - val_acc: 0.9476
Epoch 7/10
1266048/1266048 [==============================] - 289s 228us/step - los

KeyboardInterrupt: 

Давайте теперь попробуем сделать предсказание этой моделью.

In [88]:
sample = []
gold = tokenizer.tokenize(data[3])
for sent in gold:
    sample += normalize(sent)
    

In [89]:
stack = []
sents = []
for word in sample:
#     if word not in word2id:
#         continue
    stack.append(word)
    vec = [word2id.get(w, 1) for w in stack]
    vec = pad_sequences([vec], maxlen)
    pred = model.predict(vec)
    if pred[0][0] > 0.6:
        sents.append(stack)
        stack = []

In [91]:
sents[:10]

[['в',
  'ленинском',
  'районном',
  'суде',
  'продолжаются',
  'слушания',
  'по',
  'делу',
  'экс-депутата',
  'думы',
  'владивостока',
  'зинаиды',
  'ким',
  'и',
  'бывшего',
  'председателя',
  'избирательного',
  'участка',
  '№',
  '522',
  'елены',
  'футиной',
  'которых',
  'обвиняют',
  'в',
  'сговоре',
  'и',
  'фальсификациях',
  'результатов',
  'на',
  'выборах',
  'на',
  'сентябрьских',
  'выборах',
  '2016',
  'года',
  'напомним',
  '18',
  'сентября',
  '2017',
  'года'],
 ['местные',
  'журналисты',
  'сняли',
  'на',
  'видео',
  'как',
  'ким',
  'будучи',
  'кандидатом',
  'по',
  'спискам',
  'в',
  'законодательное',
  'собрание',
  'приморского',
  'края',
  'выдавала',
  'молодым',
  'людям',
  'открепительные',
  'возила',
  'их',
  'голосовать',
  'на',
  'участок',
  'где',
  'уже',
  'знали',
  'о',
  'предстоящем'],
 ['визите',
  'в',
  'качестве',
  'вознаграждения',
  'избирателям',
  'предлагалось',
  'по',
  '500',
  'рублей'],
 ['перед',
  'н

Получается так себе. Видно, что модель переобучается (ошибка на трейне уменьшается, а на валидации нет), то есть просто выучивает трейн, но очень плохо обобщает. Возможно, стоит попробовать добавить побольше данных или нарезать их получше.